In [ ]:
!pip install -qq transformers datasets comet_ml

In [1]:
# PARAMETERS

DATASET = "simecek/Human_DNA_v0_BPEtokenized_vocab30k"
HF_MODEL_NAME = "DNADebertaBPE30k"

LR = 5e-05
WD = 0.1

BATCH_SIZE = 16
ACCUMULATION = 4
HIDDEN_LAYERS = 6
MLM_P = 0.15
EPOCHS = 15

## 0) Loading

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
import os

os.environ['COMET_API_KEY'] = "uckBYzSuTQ0uUKVA42536dD7Q"

import comet_ml

# Commet Init
comet_ml.init(project_name="Training_20220714", api_key="uckBYzSuTQ0uUKVA42536dD7Q")

COMET INFO: Comet API key is valid
COMET WARNING: running in Google Colab, but can't find mounted drive. Using HOME instead
COMET WARNING: if drive is mounted, set COMET_CONFIG to save config there
COMET INFO: Comet API key saved in /root/.comet.config


In [4]:
from datasets import load_dataset
datasets = load_dataset(DATASET)
datasets.set_format("torch")
datasets

Using custom data configuration simecek--Human_DNA_v0_BPEtokenized_vocab30k-392f22544bcaf23b
Reusing dataset parquet (/root/.cache/huggingface/datasets/simecek___parquet/simecek--Human_DNA_v0_BPEtokenized_vocab30k-392f22544bcaf23b/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 936729
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 104085
    })
})

In [5]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("Vlasta/DNA_BPE_max_vocab_30000_maxk_10_max_tokenlen_10")

## 1) Training

In [6]:
from transformers import DebertaConfig, DebertaForMaskedLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

model_config = DebertaConfig(vocab_size=len(tokenizer.vocab), max_position_embeddings=512, num_hidden_layers=HIDDEN_LAYERS)
model_config

DebertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": null,
  "position_biased_input": true,
  "relative_attention": false,
  "transformers_version": "4.20.1",
  "type_vocab_size": 0,
  "vocab_size": 30000
}

In [7]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=MLM_P)

model = DebertaForMaskedLM(config=model_config)
sum(p.numel() for p in model.parameters()) // 10**6

66

In [8]:
training_args = TrainingArguments(
    output_dir='./model',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=EPOCHS,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=BATCH_SIZE, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=ACCUMULATION,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=BATCH_SIZE,  # evaluation batch size
    logging_steps=5000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=5000,
    fp16=True,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=1,           # whether you don't have much space so you let only 5 model weights saved in the disk
    push_to_hub=True,
    hub_model_id=HF_MODEL_NAME,
    hub_strategy="every_save",
    learning_rate=LR,
    weight_decay=WD
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=datasets['train'],
    eval_dataset=datasets['test'],
)

/content/./model is already a clone of https://huggingface.co/simecek/DNADebertaBPE30k. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 936729
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 219540
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `

Step,Training Loss,Validation Loss


In [ ]:
import torch 
#torch.cuda.empty_cache() 

In [ ]:
tokenizer("ACTGGGCTA")